In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.max_columns = 999
%matplotlib inline

shopify_data = pd.read_csv('orders_export_201805.csv', 
                           low_memory=False, 
                           dtype={'Lineitem sku': 'str', 'Name': 'str'},
                           parse_dates = ['Paid at', 'Created at'])

new_columns = ['Name', 'Email', 'Created at', 'Lineitem quantity', 'Lineitem name', 
               'Lineitem price', 'Lineitem sku','Location']

shopify_data_simple = shopify_data[new_columns]

FileNotFoundError: File b'orders_export_201805.csv' does not exist

In [ ]:
# pd.DataFrame(shopify_data_simple.groupby(['Email', 'Name', 'Created at', 'Lineitem sku'])['Lineitem quantity'].sum())
shopify_all_sizes = pd.DataFrame(shopify_data_simple.groupby(['Email', 'Lineitem sku', 'Lineitem name'])['Lineitem quantity'].sum()).reset_index()
shopify_data_simple['Lineitem sku'] = shopify_data_simple['Lineitem sku'].astype('str')
shopify_full_size_only = shopify_data_simple[~shopify_data_simple['Lineitem sku'].str.endswith(('6', '7', '9', '8'))]
shopify_full_size_only

In [ ]:
transaction_dates = pd.DataFrame(shopify_full_size_only.groupby(['Email', 'Created at', 'Lineitem sku', 'Lineitem name'])['Lineitem quantity'].sum()).reset_index()
transaction_dates['Latest Transaction'] = transaction_dates.groupby(['Email'])['Created at'].transform('max')
transaction_dates['Earliest Transaction'] = transaction_dates.groupby(['Email'])['Created at'].transform('min')
transaction_dates['Time as customer'] = transaction_dates['Latest Transaction']  - transaction_dates['Earliest Transaction']

In [ ]:
# type(transaction_dates['Created at'][0])
email_sku_count = pd.DataFrame(transaction_dates.groupby(['Email', 'Lineitem sku'])['Lineitem quantity'].sum()).reset_index()
email_sku_count

In [ ]:
email_sku_count[email_sku_count['Email'] == 'Dorislim3@ymail.com']

In [ ]:
emails = list(np.sort(email_sku_count['Email'].unique()))
products = list(np.sort(email_sku_count['Lineitem sku'].unique()))
quantity = email_sku_count['Lineitem quantity']

In [ ]:
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve

rows = email_sku_count['Email'].astype('category', categories = emails).cat.codes
cols = email_sku_count['Lineitem sku'].astype('category', categories = products).cat.codes
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(emails), len(products)))

In [ ]:
purchases_sparse

In [ ]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

For collaborative filtering to work, the maximum sparsity you could get away with would probably be about 99.5% or so. We are well below this, so we should be able to get decent results.

In [ ]:
import random

def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  

In [ ]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

In [ ]:
import implicit
# from implicit.als import AlternatingLeastSquares
alpha = 15
# user_vecs, item_vecs = ...
model = implicit.als.AlternatingLeastSquares(factors=20, 
                                            regularization = 0.1,
                                           iterations = 15)
model.fit(product_train*alpha)
item_vecs = model.item_factors 
user_vecs = model.user_factors

In [ ]:
from sklearn import metrics
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)   

In [ ]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

In [ ]:
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(item_vecs), sparse.csr_matrix(user_vecs.T)], product_test)
# AUC for our recommender system

### A Recommendation Example

In [ ]:
item_lookup = shopify_data_simple[['Lineitem sku', 'Lineitem name']].drop_duplicates() # Only get unique item/description pairs
item_lookup['Lineitem sku'] = item_lookup['Lineitem sku'].astype(str) # Encode as strings for future lookup ease

In [ ]:
customers_arr = np.array(emails) # Array of customer IDs from the ratings matrix
products_arr = np.array(products) # Array of product IDs from the ratings matrix

def get_items_purchased(customer_id, mf_train, customers_list, products_list, item_lookup):

    cust_ind = np.where(customers_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items
    prod_codes = products_list[purchased_ind] # Get the stock codes for our purchased items
    return item_lookup.loc[item_lookup['Lineitem sku'].isin(prod_codes)]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
    
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    pref_vec = mf_train[cust_ind,:].toarray() # Get the ratings from the training set ratings matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    # Items already purchased have their recommendation multiplied by zero
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    # of best recommendations
    rec_list = [] # start empty list to store items
    for index in product_idx:
        code = item_list[index]
        rec_list.append([code, item_lookup['Lineitem name'].loc[item_lookup['Lineitem sku'] == code].iloc[0]]) 
        # Append our descriptions to the list
    codes = [item[0] for item in rec_list]
    descriptions = [item[1] for item in rec_list]
    final_frame = pd.DataFrame({'Lineitem sku': codes, 'Lineitem name': descriptions}) # Create a dataframe 
    return final_frame[['Lineitem sku', 'Lineitem name']] # Switch order of columns around

In [ ]:
# 'Dorislim3@ymail.com'
# 'Rachelhfteoh@yahoo.com'
# 'Shirley_a_louis@yahoo.com'
customers_arr[125]

In [ ]:
rec_items('Shirley_a_louis@yahoo.com', product_train, item_vecs, user_vecs, customers_arr, products_arr, item_lookup)

In [ ]:
get_items_purchased('Shirley_a_louis@yahoo.com', product_train, customers_arr, products_arr, item_lookup)

In [1]:
import implicit